In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
result_path = "../new/results/"

In [3]:
data_export_path = "../new/results/"

In [4]:
df = pd.read_csv("output_data.csv")

/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_38347/399446867.py:1: DtypeWarning: Columns (12,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("output_data.csv")


In [5]:
# replace na with ""
df.fillna("",inplace=True)

/var/folders/16/dh6sy2v17_l39d8bcf_6y7n00000gr/T/ipykernel_38347/2091081822.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("",inplace=True)


In [6]:
abstract_df = df[['display_name','doi','abstract_text']].reset_index(drop=True)

In [7]:
# We use title and abstract, as well focus on sentence-line context aware info, not keywords.
abstract_df["Title_abstract"] = abstract_df["display_name"]+". "+abstract_df["abstract_text"]
# now titel only
abstract_df["Title_abstract"] = abstract_df["display_name"]

### Topic Modeling

In [8]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans

In [9]:
docs = [str(item) for item in list(abstract_df.Title_abstract)]

In [10]:
# [TODO 1] need to consider remove query keywords
# [TODO 2] remove non-English
# [TODO 3] cluster number: more experiments [20, 30, 40, 50, 100, 150, 200]
# 200 too many
vectorizer_model = CountVectorizer(stop_words="english")
# hdbscan_model = HDBSCAN(min_cluster_size=10,
#                         min_samples=5,
#                         metric='euclidean',
#                         cluster_selection_method='eom',
#                         prediction_data=True) # random_state=42
cluster_model = KMeans(n_clusters=200)
topic_model = BERTopic(hdbscan_model=cluster_model, #hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       top_n_words=10) # nr_topics=100,
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/lizhouf/opt/anaconda3/envs/bertopic/lib/python3.10/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [11]:
topic_model.save(result_path+"/LLM_title_200")

/Users/lizhouf/opt/anaconda3/envs/bertopic/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [12]:
abstract_df["topics"] = topics

In [13]:
abstract_df

,display_name,doi,abstract_text,Title_abstract,topics
0,Performance of ChatGPT on USMLE: Potential for...,https://doi.org/10.1371/journal.pdig.0000198,We evaluated the performance of a large langua...,Performance of ChatGPT on USMLE: Potential for...,18
1,How Does ChatGPT Perform on the United States ...,https://doi.org/10.2196/45312,Background Chat Generative Pre-trained Transfo...,How Does ChatGPT Perform on the United States ...,157
2,ChatGPT and the Future of Medical Writing,https://doi.org/10.1148/radiol.223312,"HomeRadiologyVol. 307, No. 2 PreviousNext Revi...",ChatGPT and the Future of Medical Writing,35
3,Performance of ChatGPT on USMLE: Potential for...,https://doi.org/10.1101/2022.12.19.22283643,ABSTRACT We evaluated the performance of a lar...,Performance of ChatGPT on USMLE: Potential for...,18
4,"The Role of ChatGPT, Generative Language Model...",https://doi.org/10.2196/46885,ChatGPT is a generative language model tool la...,"The Role of ChatGPT, Generative Language Model...",18
...,...,...,...,...,...
17961,Applying GPT-4 to the Plastic Surgery Inservic...,https://doi.org/10.1016/j.bjps.2023.09.027,The recent introduction of Generative Pre-trai...,Applying GPT-4 to the Plastic Surgery Inservic...,93
17962,ChatGPT for low- and middle-income countries: ...,https://doi.org/10.1016/j.lanwpc.2023.100906,"ChatGPT (OpenAI, San Francisco, CA, USA) has m...",ChatGPT for low- and middle-income countries: ...,33
17963,Interpretable Disease Prediction via Path Reas...,https://doi.org/10.1016/j.knosys.2023.111082,Disease prediction based on patients’ historic...,Interpretable Disease Prediction via Path Reas...,54
17964,ChatGPT (Generated Pre-Trained Transformer) As...,https://doi.org/10.23937/2572-4037.1510062,Mental health disorders affect one in four peo...,ChatGPT (Generated Pre-Trained Transformer) As...,74


In [14]:
# show topics info
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,278,0_chatgpt_chatgpts_medical_questions,"[chatgpt, chatgpts, medical, questions, utilit...",[Correspondence on Letter 2 regarding “Assessi...
1,1,240,1_growth_fish_dietary_juvenile,"[growth, fish, dietary, juvenile, tilapia, nil...",[Mango pulp meal in diets for Nile tilapia (Or...
2,2,237,2_ai_intelligence_artificial_climate,"[ai, intelligence, artificial, climate, cybers...","[Artificial intelligence, Is this artificial i..."
3,3,222,3_gags_gag_welcome_,"[gags, gag, welcome, , , , , , , ]","[Welcome, GAG, GAGs]"
4,4,219,4_colombia_mxico_en_el,"[colombia, mxico, en, el, la, las, salud, del,...",[Una clasificación de las modalidades de vulne...
...,...,...,...,...,...
195,195,17,195_gastrointestinaltrakt_gastrinstimulationst...,"[gastrointestinaltrakt, gastrinstimulationstes...","[Gastrin nach Sekretinstimulation, Proceedings..."
196,196,16,196_issue_information_bwhole_acknowledgments,"[issue, information, bwhole, acknowledgments, ...","[Issue Information, Issue Information, Issue I..."
197,197,15,197_covid19___,"[covid19, , , , , , , , , ]",[Клінічні настанови з оцінки потенційно злоякі...
198,198,15,198_index_author_page_geindex,"[index, author, page, geindex, eumc, educon, s...","[Index, Index, Index]"


In [15]:
topic_model.get_topic_info().to_csv(data_export_path+"/LLM_openalex_title_topics_metadata_200.csv",index=False)

In [16]:
# show Topic Positions -> suggestions for further clustering
#topic_model.visualize_topics()
fig_IDM = topic_model.visualize_topics()
fig_IDM.write_html(result_path+"/LLM_openalex_title_viz_IDM_200.html")

In [17]:
# visualize Topic Hierarchy -> suggestions for further clustering
fig_hierarchy = topic_model.visualize_hierarchy()
fig_hierarchy.write_html(result_path+"/LLM_openalex_title_viz_hierarchy_200.html")

In [18]:
# visualize words
fig_topic_word_scores = topic_model.visualize_barchart(top_n_topics=32)
fig_topic_word_scores.write_html(result_path+"/LLM_openalex_title_viz_topic_word_scores_200.html")

In [19]:
# visualize topic similarity -- not useful in our case, maybe include to show not correlated
fig_heatmap = topic_model.visualize_heatmap()
fig_heatmap.write_html(result_path+"/LLM_openalex_title_viz_heatmap_200.html")

In [20]:
# visualize documents
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# save embeddings (for Tableau)
abstract_df["x"] = reduced_embeddings[:, 0]
abstract_df["y"] = reduced_embeddings[:, 1]


In [21]:
import pickle
with open(result_path+'/LLM_openalex_title_embeddings_200.pickle', 'wb') as pkl:
    pickle.dump(embeddings, pkl)

In [22]:
abstract_df.to_csv(data_export_path+"/LLM_openalex_title_w_topics_years_positions_200.csv",index=False)